# Selecting the Index constituents

# Importing and laoding packages

In [2]:
import pandas as pd
import glob
import os
import numpy as np
import calendar
import datetime
import nltk
import feather
import pyarrow
import string
import xlrd
import re
import operator 
import math
import sklearn
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from nltk import bigrams 
from collections import Counter
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hagen\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
from collections import defaultdict
from scipy.stats import zscore

In [4]:
# np.set_printoptions(precision=4)

# Twitter data

## Load raw data from Excel

Careful, this is a large dataset. Skip this part if feather data available!

In [5]:
# OLD
# # load all raw cashtag data
# path = r'C:\Users\hagen\00_Computing\data_cashtags' # adjust path
# all_files = glob.glob(os.path.join(path, "*.csv")) # read all csv files
# df_cashtag_data = pd.concat((pd.read_csv(file)
#     .assign(ticker = os.path.splitext(os.path.basename(file))[0]) for file in all_files)) # +ticker

In [6]:
# OLD
# # load all raw hashtag data
# path = r'C:\Users\hagen\00_Computing\data_hashtags' # adjust path
# all_files = glob.glob(os.path.join(path, "*.csv")) # read all csv files
# dtypes = {"author_id": int, "username": "string", "author_followers": int, "author_following": int, "author_tweets": int,
#     "author_listed": int, "author_description": "string", "author_location": "string", "user_id": bool, 
#     "author_account_age": "string", "tweet_id": int, "text": "string", "created_at": "string", 
#     "retweets": int, "replies": int, "likes": int, "quote_count": int, "referenced_tweets": "string", "media": "string"}
# # parse_dates = ["author_account_age", "created_at"]
# df_hashtag_data = pd.concat((pd.read_csv(file, dtype=dtypes, sep=',', header=None, low_memory=False) #, parse_dates = parse_dates)
#     .assign(ticker = os.path.splitext(os.path.basename(file))[0]) for file in all_files)) # +ticker

In [7]:
# dtypes_short = {"author_followers": int, "author_following": int, "author_tweets": int,
#     "author_listed": int, "author_verified": bool, 
#     "author_account_age": str, "text": str, "created_at": str, 
#     "retweets": int, "replies": int, "likes": int, "quote_count": int}
# dtypes = {"author_id": "long", "username": str, "author_followers": int, "author_following": int, "author_tweets": int,
#     "author_listed": int, "author_description": str, "author_location": str, "author_verified": bool, 
#     "author_account_age": str, "tweet_id": "long", "text": str, "created_at": str, 
#     "retweets": int, "replies": int, "likes": int, "quote_count": int, "referenced_tweets": str, "media": str}

In [50]:
# # import cashtag data from Excel and save to feather
# cashtag_xls = pd.ExcelFile('data_cashtags_all.xlsx')
# df_cashtag_data = pd.read_excel(cashtag_xls, dtype=dtypes_short, usecols="C:F,I:J,L:Q,T")
# cashtag_pd_filepath = "./cashtag_pd.ftr"
# df_cashtag_data.to_feather(cashtag_pd_filepath)

In [63]:
# # import hashtag data from Excel and save to feather
# hashtag_xls = pd.ExcelFile('data_hashtags_combined.xlsx')
# hashtag_sheets = hashtag_xls.sheet_names
# df_hashtag_data = pd.concat((pd.read_excel(hashtag_xls, sheet_name=s, dtype=dtypes_short, usecols="C:F,I:J,L:Q,T")
#                              .assign(ticker=s) for s in hashtag_sheets)
#                              , ignore_index=True)
# hashtag_pd_filepath = "./hashtag_pd.ftr"
# df_hashtag_data.to_feather(hashtag_pd_filepath)

# Data preparation

## Tokenization pipeline

In [575]:
# define emoticons
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""

# define other regular expressions
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r"(?:\$+[\w_]+[\w\'_\-]*[\w_]+)", # cash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
    
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    # check for alternative 
    # https://stackoverflow.com/questions/43142710/remove-all-punctuation-from-string-except-if-its-between-digits
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]

tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE) #consider adding cashtags

In [576]:
sw_twitter = ['rt', 'via']
sw_company = ['ag', 'gmbh', 'se']
sw_hype = ['#crypto', '#cryptocurrency', '#decentralisation', '#defi', 'blockchain', '#btc', '#dragonbite', '#bitmart', '#eth', '#nfts', '#nft']
sw_other = ['’', "i'm", 'therefore'] # add larger stop word dictionary than NLTK?
sw_spam = ['#swaap', '@swaapcrypto', 'swaap', '#cannabis']

In [577]:
# for removing stop-words
punctuation = list(string.punctuation)
stop = stopwords.words('english') + punctuation + sw_twitter + sw_company + sw_crypto + sw_other + sw_spam

In [578]:
def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=True):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens

In [579]:
# what about term.lower()?
def tweet_cleaning(tweet):
    tweet = preprocess(tweet)
    tweet_clean = [term for term in tweet 
                  if term not in stop and
                  not term.startswith(('@', '$', 'http', '_', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9'))]
    # consider removing numbers, and all mentions, incl. #, @, and $, as well as all links
    return tweet_clean

In [580]:
# # consider removing hashtags
# terms_hash = [term for term in list_clean if term.startswith('#')]
# # consider removing hashtags
# terms_cash = [term for term in list_clean if term.startswith('$')]
# # removes all types of tags
# terms_only = [term for term in list_clean if not term.startswith(('#', '@', '$'))] 
# # terms_only = [term for term in preprocess(tweet['text']) 
# #               if term not in stop and
# #               not term.startswith(('#', '@'))]

## Choosing data to  work on 

### Example

In [581]:
# # read example data with two tickers
# ct_short = pd.read_excel('data_cashtags_short.xlsx', usecols="C:F,I:J,L:Q,T")

### Feather

In [582]:
# read actual data from feather

Cashtag data

In [595]:
# read cashtag data from feather
cashtag_pd_filepath = "./cashtag_pd.ftr"
ct_cashtags = pd.read_feather(cashtag_pd_filepath, columns=None, use_threads=True)

Hashtag data -! this is a large dataset

In [596]:
# read hashtag data from feather
hashtag_pd_filepath = "./hashtag_pd.ftr"
ct_hashtags = pd.read_feather(hashtag_pd_filepath, columns=None, use_threads=True)
# replace all additional tickers
ct_hashtags["ticker"].replace({"MBGn_other": "MBGn", "BMWG_other": "BMWG", "ADSGn_other": "ADSGn", "ADSGn_media": "ADSGn"}, inplace=True)
ct_hashtags.drop(columns=['author_location', 'username', 'tweet_id', 'media'], inplace=True)

In [613]:
ct_short = ct_hashtags.append(ct_cashtags, ignore_index=True)

In [614]:
len(ct_cashtags)

187253

In [615]:
len(ct_hashtags)

7568280

## Cleaning data

In [616]:
len(ct_short)

7755533

In [617]:
# drop exactly identical tweets
ct_short = ct_short.drop_duplicates(subset=['text'])
ct_short.reset_index(drop=True, inplace=True)
len(ct_short)

7351585

In [618]:
ct_short["interaction"] = ct_short["retweets"]+ct_short["replies"]+ct_short["likes"]+ct_short["quote_count"]

In [619]:
ct_short.drop(ct_short.loc[ct_short.interaction == 0].index, inplace=True)
ct_short.reset_index(drop=True, inplace=True)
len(ct_short)

3780169

In [620]:
ct_short.drop(ct_short.loc[ct_short.author_account_age > '2020-09-18'].index, inplace=True)
ct_short.reset_index(drop=True, inplace=True)
len(ct_short)

3734840

In [621]:
ct_short.drop(ct_short.loc[ct_short.author_followers < 100].index, inplace=True)
ct_short.reset_index(drop=True, inplace=True)
len(ct_short)

3165026

In [622]:
ct_short.drop(ct_short.loc[ct_short.author_tweets < 100].index, inplace=True)
ct_short.reset_index(drop=True, inplace=True)
len(ct_short)

3155410

In [623]:
ct_short.drop(ct_short.loc[ct_short.author_following == 0].index, inplace=True)
ct_short.reset_index(drop=True, inplace=True)
len(ct_short)

3130859

In [624]:
# extract tickers
# ticker_list = ["1COV", "BEIG"]
ticker_list = pd.unique(ct_short["ticker"]).tolist()
# for full list, change all names to unique tickers for Adidas, MB, VW, BMW

In [625]:
ticker_list

['WDIG',
 'VOWG_p',
 'VNAn',
 'TKAG',
 'SIEGn',
 'SAPG',
 'RWEG',
 'PSMGn',
 'MUVGn',
 'MTXGn',
 'MRCG',
 'MBGn',
 'LINI',
 'LHAG',
 'IFXGn',
 'HNKG_p',
 'HEIG',
 'FREG',
 'FMEG',
 'EONGn',
 'ENR1n',
 'DWNG',
 'DTEGn',
 'DPWGn',
 'DHER',
 'DBKGn',
 'DB1Gn',
 'CONG',
 'CBKG',
 'BMWG',
 'BEIG',
 'BAYGn',
 'BASFn',
 'ALVG',
 'ADSGn',
 '1COV']

In [626]:
# clean data
list_short = ct_short["text"].apply(tweet_cleaning)
# this is the cleanest data and will be used going forward

In [631]:
# remove duplicates, i.e. each token appears only once
list_single = list_short.copy()
list_single = list_single.apply(set)

In [632]:
# flatten set
list_single_flat = [item for sublist in list_single for item in sublist]

In [633]:
# # creates one list out of nested lists (flattens nested lists), needed to count items
# list_short_flat = list_short.sum()
# # count most common items
# Counter(list_short_flat).most_common(10)

In [634]:
# # creat bigrams and count
# terms_bigram = bigrams(list_short_flat)
# # Counter(terms_bigram).most_common(10)

# TBD: including bigrams instead of only single terms

In [635]:
# create co-occurence matrix
com = defaultdict(lambda : defaultdict(int))
terms_only = []
for x in range(len(list_short)):
    terms_only = [term for term in list_short[x]]
    
    for i in range(len(terms_only)-1):            
        for j in range(i+1, len(terms_only)):
            w1, w2 = sorted([terms_only[i], terms_only[j]])                
            if w1 != w2:
                com[w1][w2] += 1

In [636]:
# export co-occurence terms
com_max = []
# For each term, look for the most common co-occurrent terms
for t1 in com:
    t1_max_terms = sorted(com[t1].items(), key=operator.itemgetter(1), reverse=True)[:5]
    for t2, t2_count in t1_max_terms:
        com_max.append(((t1, t2), t2_count))
# Get the most frequent co-occurrences
# terms_max = sorted(com_max, key=operator.itemgetter(1), reverse=True)
# print(terms_max[:5])

In [637]:
# count single terms
count_all = Counter()
count_all.update(list_single_flat)

In [638]:
# calculating probabilities
# n_docs is the total n. of tweets
n_docs = len(list_short)
p_t = {}
p_t_com = defaultdict(lambda : defaultdict(int))
 
for term, n in count_all.items():
    p_t[term] = n / n_docs
    for t2 in com[term]:
        p_t_com[term][t2] = com[term][t2] / n_docs

In [639]:
# import vocab
wb_pos = xlrd.open_workbook("dic_positive.xlsx")
ws_pos = wb_pos.sheet_by_index(0)
positive_vocab = ws_pos.col_values(0)
positive_vocab = [x.lower() for x in positive_vocab]
wb_neg = xlrd.open_workbook("dic_negative.xlsx")
ws_neg = wb_neg.sheet_by_index(0)
negative_vocab = ws_neg.col_values(0)
negative_vocab = [x.lower() for x in negative_vocab]

In [640]:
# # insert NLP dictionary
# positive_vocab = [
#     'good', 'nice', 'great', 'awesome', 'outstanding',
#     'fantastic', 'terrific', ':)', ':-)', 'like', 'love',
#     # shall we also include game-specific terms?
#     # 'triumph', 'triumphal', 'triumphant', 'victory', etc.
# ]
# negative_vocab = [
#     'bad', 'terrible', 'crap', 'useless', 'hate', ':(', ':-(',
#     # 'defeat', etc.
# ]

In [ ]:
# calculate PMI
pmi = defaultdict(lambda : defaultdict(int))
for t1 in p_t:
    for t2 in com[t1]:
        denom = p_t[t1] * p_t[t2]
        pmi[t1][t2] = math.log2(p_t_com[t1][t2] / denom)
 
semantic_orientation = {}
for term, n in p_t.items():
    positive_assoc = sum(pmi[term][tx] for tx in positive_vocab)
    negative_assoc = sum(pmi[term][tx] for tx in negative_vocab)
    semantic_orientation[term] = positive_assoc - negative_assoc

In [ ]:
# show top and bottom PMI tokens
semantic_sorted = sorted(semantic_orientation.items(), 
                         key=operator.itemgetter(1), 
                         reverse=True)
# top_pos = semantic_sorted[:10]
# top_neg = semantic_sorted[-10:]
 
# print(top_pos)
# print(top_neg)

In [ ]:
# save PMI into df
scores = pd.DataFrame(semantic_sorted, columns = ['token', 'score'])

In [ ]:
scores_nonzero = scores[scores['score'] != 0]

In [ ]:
scores_nonzero.to_excel("scores_nonzero.xlsx")

# TBD: removing all tokes with https://, coins etc.

In [491]:
# print top 20
scores.head(20)

,token,score
0,#id4,77.927454
1,awesome,67.350624
2,concept,65.676239
3,#jha,54.831341
4,#dtcmd18,53.630986
5,artificial,49.144920
6,#china's,49.039119
7,#faw,48.670953
8,overall,46.498405
9,analytics,46.166089


In [492]:
# print bottom 20
scores.tail(20)
# why do $db and $DB?

,token,score
34785,db,-234.150361
34786,fed,-234.679420
34787,billion,-236.108887
34788,another,-236.928835
34789,derivatives,-237.177665
34790,banking,-243.845325
34791,could,-246.733095
34792,assets,-255.613674
34793,case,-263.037148
34794,due,-265.071527


In [493]:
# vectorize
vectorizer = CountVectorizer(analyzer = tweet_cleaning, dtype = 'uint8')
tweets_countvectorizer = vectorizer.fit_transform(ct_short["text"])

In [494]:
# calculate scores for each tweet
vectorizer_tokens = vectorizer.get_feature_names()
vectorizer_tokens = pd.DataFrame(vectorizer_tokens, columns = ['token'])
combined = pd.merge(vectorizer_tokens, scores, on="token", how="left")
# fill potential NaN with zero
combined['score'] = combined['score'].fillna(0)
combined = combined['score'].drop(columns=['token'])
# b = np.array(combined)
b = np.matrix(combined).transpose()
c = (tweets_countvectorizer *b)

In [495]:
# a = tweets_countvectorizer.toarray()
# c = a*b
# c = np.sum(c, axis=1)
# # print scores and tokens
# print(scores.score.to_string())
# print(scores.token.to_string())

# Combining scores with date and ticker data

In [496]:
# take only date, ticker, and score
tweets_scored = ct_short[['created_at', 'ticker']].copy()
sum_score = pd.DataFrame(c)
tweets_scored["score"] = sum_score.copy()

In [497]:
# group by date and ticker 
# https://stackoverflow.com/questions/35898667/group-by-time-and-other-column-in-pandas
tweets_scored['date'] = pd.to_datetime(tweets_scored['created_at'])
tweets_scored.index = tweets_scored['date']
tweets_scored_bydate = tweets_scored.groupby([pd.Grouper(freq='D'), "ticker"]).mean()
# # this creates multiindex
# tweets_scored_bydate.index
# tweets_scored_bydate.index.nlevels
# https://stackoverflow.com/questions/49395100/change-pandas-multi-index-from-row-to-column
tweets_scored_bydate = tweets_scored_bydate.unstack()
tweets_scored_bydate.columns = tweets_scored_bydate.columns.droplevel(0)
# print(tweets_scored_bydate.to_string())

In [498]:
# tweets_scored_bydate.loc["2018-02-19"]

In [499]:
# create new dataframe with all dates for subsequent calcs
all_dates = pd.date_range(start="2016-01-01",end="2021-12-31")
all_dates_df = pd.DataFrame(all_dates, columns=["date"])
all_dates_df.index = all_dates_df['date']
#all_dates_df["all_dates"] = all_dates_df['date'] # not needed
all_dates_df = all_dates_df.drop('date', 1)
#all_dates_df["ticker"] = "" # not needed
#print(all_dates_df.to_string())

In [500]:
# # create multiindex for second df
# all_dates_df['ticker'] = ''
# all_dates_df.set_index('ticker', append=True, inplace=True)
# all_dates_df.index.nlevels

In [501]:
# merge into one and fill NaN
scoring = pd.merge(all_dates_df, tweets_scored_bydate, on=["date"], how="left")
scoring = scoring.fillna(0)
scoring["decay_factor"] = 0.9
#print(scoring.to_string())

In [502]:
# scoring.loc["2018-02-19"]

# Calculate EWA scores

In [503]:
# # calculate the last Friday per month for rebalancing
# def rebalancing_dates():
#     fridays = []
#     for year in range (2016, 2022):
#         for month in range(1, 13):
#             last_friday = max(week[calendar.FRIDAY]
#                 for week in calendar.monthcalendar(year, month))
#             fridays.append(str('{:4d}-{:02d}-{:02d}'.format(year, month, last_friday)))
#     return fridays
# #rebalancing_dates()

In [504]:
# generate 3rd Friday each month
friday3_list = pd.date_range('2016-01-01','2021-12-31',freq='WOM-3FRI').format()
# friday3_df = pd.DataFrame(friday3_list)
# friday3_df.to_csv('rebalancing_dates.csv')

In [505]:
# 19.04.2019 not a trading day, "Karfreitag", now moved to next weekday, which is Tuesday
friday3_list[friday3_list.index("2019-04-19")] = '2019-04-23'

In [506]:
# calculate EWA as per S&P methodology
ewa = pd.DataFrame(friday3_list, columns=["date"])
ewa["next_rb"] = ewa["date"].copy()
ewa["last_rb"] = ewa["next_rb"].copy()
ewa["last_rb"] = ewa["last_rb"].shift(+1)
ewa["next_rb"] = pd.to_datetime(ewa["next_rb"])
ewa["last_rb"] = pd.to_datetime(ewa["last_rb"])
ewa["N"] = abs((ewa["last_rb"] - ewa["next_rb"]).dt.days)
ewa.drop(ewa.tail(1).index,inplace=True)
ewa['date'] = pd.to_datetime(ewa['date'])
ewa.index = ewa['date']
# drop second date column
ewa = ewa.drop('date', 1)

In [507]:
# ewa.to_excel("00_ewa.xlsx")
# scoring.to_excel("00_scoring.xlsx")
# scoring_ewa.to_excel("00_scoring_ewa.xlsx")

In [508]:
# calculate alpha
scoring_ewa = pd.merge_asof(scoring, ewa, on='date', direction='forward')
scoring_ewa["t"] = abs(((scoring_ewa["next_rb"] - scoring_ewa["date"]).dt.days) - scoring_ewa["N"])
scoring_ewa["a"] = (scoring_ewa["decay_factor"]**(scoring_ewa["N"] - scoring_ewa["t"]))
#print(scoring_ewa.to_string())
scoring_ewa.index = scoring_ewa["date"]
scoring_ewa['rb'] = np.where(scoring_ewa['date']==scoring_ewa['next_rb'], True, False)
# drop second date column
scoring_ewa = scoring_ewa.drop('date', 1)
scoring_ewa = scoring_ewa[scoring_ewa['t'].notna()]

In [509]:
# print(scoring_ewa.to_string())

In [510]:
# multiply all ticker columns by column a
scoring_ewa_a = scoring_ewa.copy()
scoring_ewa_a = scoring_ewa.apply(lambda x: x*scoring_ewa["a"] if x.name in ticker_list else x)

In [511]:
# scoring_ewa_a.to_excel("00_scoring_ewa_a.xlsx")

In [512]:
# print(scoring_ewa_a.to_string())

In [513]:
# # Extend to all columns
# # https://stackoverflow.com/questions/48103845/python-pandas-sum-values-in-columns-if-date-between-2-dates
# scoring_ewa["sum"] = scoring_ewa.apply(lambda x: scoring_ewa.loc[(scoring_ewa.index >= x.last_rb) & 
#                                             (scoring_ewa.index <= x.next_rb), "1COV"].sum(), axis=1)
# # maybe create a new file with just the rebalancing dates? maybe easier for index calculation?
# # may need a matrix of 0 and 1 in the end for each date and/or each trading date

In [514]:
# This works on all columns, replacing original values
# https://stackoverflow.com/questions/48103845/python-pandas-sum-values-in-columns-if-date-between-2-dates
scoring_ewa_sum = scoring_ewa_a.copy()
scoring_ewa_sum[ticker_list] = scoring_ewa_sum.apply(lambda x: scoring_ewa_sum.loc[(scoring_ewa_sum.index > x.last_rb) & 
                                            (scoring_ewa_sum.index <= x.next_rb), ticker_list].sum(), axis=1)
# maybe create a new file with just the rebalancing dates? maybe easier for index calculation?
# may need a matrix of 0 and 1 in the end for each date and/or each trading date

In [515]:
# scoring_ewa_sum.to_excel("00_scoring_ewa_sum.xlsx")

In [516]:
# print(scoring_ewa_sum.to_string())

In [517]:
# select only rows where rb = TRUE
scoring_ewa_rb = scoring_ewa_sum[scoring_ewa_sum['rb'] == True]

In [518]:
# print(scoring_ewa_rb.to_string())

In [519]:
# select only data columns
ewa_data = scoring_ewa_rb[np.intersect1d(scoring_ewa_rb.columns, ticker_list)]

In [520]:
# ewa_data.to_excel("00_ewa_data.xlsx")

In [521]:
# print(ewa_data["1COV"].to_string())

# Start here

In [522]:
# filter out companies not in benchmark
# multiply by matrix of zero and 1
xls = pd.ExcelFile('daxk_data.xlsx')
ticker_matrix_daxk = pd.read_excel(xls, 'ticker_matrix')
ticker_matrix_daxk.set_index(ticker_matrix_daxk.columns[0], inplace=True)
ticker_matrix_daxk.index = pd.to_datetime(ticker_matrix_daxk.index)
# isin_list = pd.read_excel(xls, 'isin_list')
ticker_to_isin = pd.read_excel(xls, 'ticker_to_isin')
isin_shell = pd.read_excel(xls, 'isin_shell')
isin_shell.set_index(isin_shell.columns[0], inplace=True)
isin_shell.index = pd.to_datetime(isin_shell.index)

In [523]:
ewa_data_nn = ewa_data.copy()
ewa_data_nn = ewa_data_nn.rename(columns=(ticker_to_isin.set_index('primary_ticker')['ISIN'].to_dict()))

In [524]:
ewa_data_nn.drop(ewa_data_nn.loc[ewa_data_nn.index > '2021-09-17'].index, inplace=True)
ewa_data_nn.drop(ewa_data_nn.loc[ewa_data_nn.index < '2016-09-12'].index, inplace=True)

In [525]:
# rb_dates = ewa_data_nn.index.to_list()

In [526]:
# remove dates
ticker_matrix_daxk_shortd = ticker_matrix_daxk[ticker_matrix_daxk.index.isin(ewa_data_nn.index)]

In [527]:
# print(ticker_matrix_daxk_shortd["DE000CBK1001"].to_string())

In [528]:
# remove columns
ticker_matrix_daxk_shortc = ticker_matrix_daxk[ticker_matrix_daxk.columns[ticker_matrix_daxk.columns.isin(ewa_data_nn.columns)]]

In [529]:
# remove dates and columns
ticker_matrix_daxk_shortd_shortc = ticker_matrix_daxk_shortd[ticker_matrix_daxk_shortd.columns[ticker_matrix_daxk_shortd.columns.isin(ewa_data_nn.columns)]]

In [530]:
# ticker_matrix_daxk_shortshort.to_csv('ticker_matrix_daxk_shortd_shortc.csv')
# ewa_data_nn.to_csv('ewa_data_nn.csv')

In [531]:
# print(ticker_matrix_daxk_shortd_shortc["DE0006062144"].to_string())

# Check here for correct companies: Covestro/Pro7, Wirecard/Combank, SiemensE/Beiersdorf etc.

In [532]:
# tbd offsetting ising matrix by 1? i.e. Wirecad included as per 24.09.2018
# i.e. matrix is 0 on Friday 21.09. but included from Monday on
# does this hold for exclusions too? i.e. check Commerzbank kicked out
data_np = ticker_matrix_daxk_shortd_shortc*ewa_data_nn
# print(data_np["DE0007472060"].to_string())
# turn isin matrix values into NaN not zero through multiplication by multiplying by NaN

In [533]:
# data_np.to_excel("00_data_np.xlsx")

In [534]:
# print(data_np["DE0007472060"].to_string())

In [535]:
# # same as ewa_data
# print(data_np["DE0006062144"].to_string())

In [536]:
ewa_data_np = data_np.to_numpy()
# data_np

In [537]:
# print(data_np["DE0007472060"].to_string())

In [538]:
# print(data_np["DE000CBK1001"].to_string())

In [539]:
# data_np[36]
# zscore(data_np[36])

In [540]:
# np.isfinite(data_np).all()

In [541]:
# compute z-scores across companies
ewa_data_zscore = zscore(ewa_data_np,axis=1,nan_policy='omit', ddof=1)

In [542]:
# transform z-scores to make all positive
ewa_data_zscore = np.where(ewa_data_zscore<0,(1/(1-ewa_data_zscore)),(ewa_data_zscore+1))

In [543]:
# combine into dataframe
df_ewa_data_zscore = pd.DataFrame(ewa_data_zscore, columns = data_np.columns)
df_ewa_data_zscore.index = data_np.index
# print(df_ewa_data_zscore["DE0007472060"].to_string())

In [544]:
# df_ewa_data_zscore.to_excel("00_df_ewa_data_zscore.xlsx")

In [545]:
# print(df_ewa_data_zscore["DE0007472060"].to_string())

In [546]:
# print(df_ewa_data_zscore["DE0007472060"].to_string())

In [547]:
# print(df_ewa_data_zscore["DE0006062144"].to_string())

In [548]:
# print(df_ewa_data_zscore["DE0005190003"].to_string())

In [549]:
# ticker_matrix.loc["2018-09-21"]

In [550]:
# # checks for median, also works with other cut offs
# ticker_matrix = df_ewa_data_zscore.gt(df_ewa_data_zscore.median(axis=1), axis=0).astype(int)

In [551]:
# n-largest
ticker_matrix = df_ewa_data_zscore.apply(pd.Series.nlargest, axis=1, n=15)

In [552]:
# ticker_matrix.to_excel("00_ticker_matrix.xlsx")

In [553]:
ticker_matrix = ticker_matrix.fillna(0)
ticker_matrix[ticker_matrix > 0] = 1

In [554]:
# remove dates
isin_shell_shortd = isin_shell[isin_shell.index.isin(ewa_data_nn.index)]

In [555]:
# remove columns
isin_shell_shortc = isin_shell[isin_shell.columns[~isin_shell.columns.isin(isin_shell.columns)]]

In [556]:
# columns not in ticker_matrix but in the shell
# isin_shell_shortd.columns[~isin_shell_shortd.columns.isin(ticker_matrix.columns)]

In [557]:
ticker_matrix_shortd = pd.merge(ticker_matrix, isin_shell[isin_shell_shortd.columns[~isin_shell_shortd.columns.isin(ticker_matrix.columns)]], on='date', how='left')

In [558]:
# print(ticker_matrix_shortd["DE0006062144"].to_string())

In [559]:
ticker_matrix_full = pd.merge_asof(isin_shell_shortc, ticker_matrix_shortd, on='date', direction='backward', allow_exact_matches=False)

In [560]:
ticker_matrix_full.set_index(ticker_matrix_full.columns[0], inplace=True)

In [561]:
# print(ticker_matrix_shortd["DE0007472060"].to_string())

In [562]:
# print(ticker_matrix_shortd["DE0006062144"].to_string())

In [563]:
# print(ticker_matrix_full["DE0006062144"].to_string())

In [564]:
# add 1 for all companies included from 19.09.2016 onwards
ticker_matrix_full.loc[ticker_matrix_full.index == '2016-09-12'] = 1
ticker_matrix_full.loc[ticker_matrix_full.index == '2016-09-13'] = 1
ticker_matrix_full.loc[ticker_matrix_full.index == '2016-09-14'] = 1
ticker_matrix_full.loc[ticker_matrix_full.index == '2016-09-15'] = 1
ticker_matrix_full.loc[ticker_matrix_full.index == '2016-09-16'] = 1

In [565]:
ticker_matrix_full.index = pd.to_datetime(ticker_matrix_full.index)
ticker_matrix_full = ticker_matrix_full.fillna(0)

In [566]:
# ticker_matrix.to_csv('ticker_matrix.csv')
ticker_matrix_full.to_csv('ticker_matrix_full.csv')

# Testing

In [450]:
# # # EXAMPLE
# import pandas as pd
# df = pd.DataFrame({'data': [5, 10, 20, 25, 20, 24, 16, 12],
#                    'next': ['2021-02-26', '2021-02-26', '2021-02-26', '2021-03-26', '2021-03-26', '2021-03-26', '2021-03-26', '2021-03-26'],
#                    'last': ['2020-01-29', '2020-01-29', '2020-01-29', '2021-02-26', '2021-02-26', '2021-02-26', '2021-02-26', '2021-02-26'],
#                    't': [26, 27, 28, 1, 2, 3, 4, 5],
#                    'expected_sum': ["NaN", "NaN", "NaN", 25, 45, 69, 85, 97]},
#                 index = ['2021-02-24', '2021-02-25', '2021-02-26', '2021-02-27', '2021-02-28', '2021-03-01', '2021-03-02', '2021-03-03'])

In [668]:
# # EXAMPLE
# import pandas as pd
# df_input = pd.DataFrame({'row1': [5, 10, 20], 'row2': [1, 30, 40],},
#                         index = ['2021-02-24', '2021-02-25', '2021-02-26'])
# df_expected_output = pd.DataFrame({'row1': [1, 0, 0], 'row2': [0, 1, 1],},
#                         index = ['2021-02-24', '2021-02-25', '2021-02-26'])
# df_median = df_input.median(axis=1)

In [421]:
array_obs = np.array([[ 0.8413,  0.8740,  0.3426,  0.8064],
              [ 0.9417,  0.5770,  0.2706,  0.6569],
              [ 0.1436,  0.3041,  0.9579,  0.4604],
              [ 0.8195,  0.8496,  0.409 ,  0.1273],
              [ 0.1290,  0.1842,  0.8811,  0.6631]])

In [422]:
array_obs

array([[0.8413, 0.874 , 0.3426, 0.8064],
       [0.9417, 0.577 , 0.2706, 0.6569],
       [0.1436, 0.3041, 0.9579, 0.4604],
       [0.8195, 0.8496, 0.409 , 0.1273],
       [0.129 , 0.1842, 0.8811, 0.6631]])

In [412]:
# column
array_obs[:,[0]]

array([[0.8413],
       [0.9417],
       [0.1436],
       [0.8195],
       [0.129 ]])

In [400]:
# row 
array_obs[0]

array([0.8413, 0.874 , 0.3426, 0.8064])

In [402]:
# axis = 1 is equal to rowise
(array_obs[0][0] - array_obs[0].mean())/array_obs[0].std()

0.5772150421492275

In [403]:
zscore(array_obs,axis=1)

array([[ 0.57721504,  0.72794319, -1.7215044 ,  0.41634617],
       [ 1.38134372, -0.14455679, -1.4265308 ,  0.18974387],
       [-1.05861834, -0.53242372,  1.61104073, -0.01999867],
       [ 0.89266939,  0.99287207, -0.47388211, -1.41165936],
       [-1.05504269, -0.88137829,  1.31113476,  0.62528622]])

In [413]:
# axis = 0
(array_obs[0][0] - array_obs[:,[0]].mean())/array_obs[:,[0]].std()

0.7384217501663388

In [414]:
zscore(array_obs) # default is 0

array([[ 0.73842175,  1.1330645 , -0.79757591,  1.12104701],
       [ 1.01684125,  0.06886819, -1.04764325,  0.4852001 ],
       [-1.19637191, -0.90897414,  1.33945795, -0.35054517],
       [ 0.67796811,  1.04563558, -0.56695824, -1.76727163],
       [-1.2368592 , -1.33859413,  1.07271945,  0.51156967]])